This notebook will go through using some functions I wrote in KT.py to generate a final fixed csv from the Agate fixed populations. 

The Agate project is meant to produce populations of LISA binaries that include mass transferring systems, which have been excluded from other COSMIC galaxy populations. While the eventual goal is a whole galaxy population, right now we are working with 9 fixed populationis: 3 for the bulge, 3 for the thin disk, and 3 for the thick disk. It should be noted that this is not really representative because each component contains different amounts of mass! Specifically, the bulge has a mass of $8.9x10^9 M_{\odot}$, the thin disk has a mass of $4.32x10^{10} M_{\odot}$, and the thick disk has a mass of $1.44x10^{10} M_{\odot}$. This is a problem for when we generate the full galaxy though–no big deal right now. 

The fixed populations are generated using cosmic-pop (https://cosmic-popsynth.github.io/docs/stable/fixedpop/index.html#inputs). Once installed, this can be run using:

cosmic-pop --inifile buildFiles/bulge_Params.ini --final-kstar1 10 12 --final-kstar2 10 12 -n 4

Where --inifile gives the path to a .ini file, which provides information for how binaries should be evolved, convergence information, sampling models, and filters. It also gives a random seed to be used: this should be changed each time cosmic-pop is run if we want different populations! Additionally, if we want to sample the stellar formation history correctly, we need to change SF_start (10000 for the bulge, 11000 for the thick disk, and 10000 for the thin disk) and SF_duration (1000 for the bulge, 1000 for the thick disk, and 10000 for the thin disk). The parameters --final-kstar1 and --final-kstar2 tell COSMIC a range of systems to keep: 10-12 keeps all the double white dwarf systems. Finally, -n tells COSMIC how many threads to used. I have found using four is the fastest on the Barnes 262 iMacs. 

Once the fixed populations have been generated, we want to concat them together. I have saved them on my computer in the directory Agate/Fixed_Pops, which has the subdirectories Bulge, thinDisk, and thickDisk. While any Fixed_Pop directory may be passed to KT.csv_from_fix_pops and KT.final_csv_from_fixe_pops, the subdirectory structure and names are assumed.

In [14]:
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import os 
%aimport KT 
import time
import warnings
warnings.filterwarnings('ignore')

In [16]:
help(KT.csv_from_fix_pops)

Help on function csv_from_fix_pops in module KT:

csv_from_fix_pops(base_dir, out_name='agate.csv')
    Take as input a directory which contains sub directories 'Bulge', 'thickDisk', and 'thinDisk' which each contain one or more fixed pop .h5 files. Take the bcm df from each of these files, concat them all together, and save them to a csv!
    
    base_dir: directory in which the fixed subdirectories 'Bulge', 'thickDisk, and thinDisk' are stored
    out_name: name of file to save to.
    
    Returns: nothing



In [17]:
help(KT.final_csv_from_fix_pops)

Help on function final_csv_from_fix_pops in module KT:

final_csv_from_fix_pops(base_dir, out_name='final_agate.csv', verbose=False)
    Similar to csv_from fix_pops, except uses the get_final_fix function instead of the bcm df for the fixed populations. This will take longer to run, but include information about the final evol_type.
    
    base_dir: directory in which the fixed subdirectories 'Bulge', 'thickDisk, and thinDisk' are stored
    out_name: name of file to save to.
    verbose: print stuff out as it goes
    
    Returns: nothing



In [18]:
#So, we can use either one of these. Lets use final_csv_from_fix_pops
fix_dir = os.path.join('Agate', 'Fixed_Pops')
t = time.time()
KT.final_csv_from_fix_pops(fix_dir,verbose=True)
print(f'Took: {time.time() - t} seconds to run!')

[*] found h5 file: Agate/Fixed_Pops/Bulge/dat_kstar1_10_12_kstar2_10_12_SFstart_10000_SFduration_1000_metallicity_0.017_1.h5
[*] Took 715.7397229671478 to run get_final_fix
[*] found h5 file: Agate/Fixed_Pops/Bulge/dat_kstar1_10_12_kstar2_10_12_SFstart_10000_SFduration_1000_metallicity_0.017_2.h5
[*] Took 614.9339072704315 to run get_final_fix
[*] found h5 file: Agate/Fixed_Pops/Bulge/dat_kstar1_10_12_kstar2_10_12_SFstart_10000_SFduration_1000_metallicity_0.017_3.h5
[*] Took 564.7693121433258 to run get_final_fix
[*] found h5 file: Agate/Fixed_Pops/thickDisk/dat_kstar1_10_12_kstar2_10_12_SFstart_11000_SFduration_1000_metallicity_0.00255_2.h5
[*] Took 1086.7684319019318 to run get_final_fix
[*] found h5 file: Agate/Fixed_Pops/thickDisk/dat_kstar1_10_12_kstar2_10_12_SFstart_11000_SFduration_1000_metallicity_0.00255_3.h5
[*] Took 665.238343000412 to run get_final_fix
[*] found h5 file: Agate/Fixed_Pops/thickDisk/dat_kstar1_10_12_kstar2_10_12_SFstart_11000_SFduration_1000_metallicity_0.002

So, about an hour and a half. Not ideal, and we should not be running this on hour galaxy for sure.